# DATSCI W261 ASSIGNMENT 3
Section 2<br/>
Student: Kuan Lin<br/>
Date: 1/29/2016

* <b>HW3.0. What is a merge sort? Where is it used in Hadoop? How is  a combiner function in the context of Hadoop? Give an example where it can be used and justify why it should be used in the context of this problem. What is the Hadoop shuffle?</b>

to do...

* <b>HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress). Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc.</b>

In [1]:
%%writefile mapper.py
#!/usr/bin/python
# mapper to emit counts for the three defined categories
import sys
for line in sys.stdin:    
    line = line.strip().lower()
    if line == '' or line.startswith('complaint id'): continue
        
    # get the complaint category
    category = line.split(',')[1].strip()
    counter_name = 'other_complaints'
    if category == 'debt collection':
        counter_name = 'debt_collection_complaints'
    elif category == 'mortgage':
        counter_name = 'mortgage_complaints'
    
    # update hadoop counters:
    sys.stderr.write("reporter:counter:customer_complaints,%s,1\n"%counter_name)
    # emit regular mapper result
    print '%s\t%s' % (counter_name, 1)

Writing mapper.py


In [11]:
%%writefile reducer.py
#!/usr/bin/python

import sys

current_category = None
current_count = 0
category = None
for line in sys.stdin:
    category, count = line.strip().split('\t', 1)
    if category == current_category:
        current_count += int(count)
    else:
        if current_category != None:
            # write result to STDOUT
            print '%s\t%s' % (current_category, current_count)
        current_category = category
        current_count = int(count)
if category == current_category:
    print '%s\t%s' % (current_category, current_count)

Overwriting reducer.py


In [12]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_1_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -mapper mapper.py -file /data/w261/hw3/mapper.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/Consumer_Complaints.csv -output /w261/hw3/hw3_1_output;

16/01/30 01:24:08 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_1_output
16/01/30 01:24:10 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob5504775449930135467.jar tmpDir=null
16/01/30 01:24:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 01:24:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 01:24:18 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 01:24:18 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 01:24:19 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454112721658_0005
16/01/30 01:24:19 INFO impl.YarnClientImpl: Submitted application application_1454112721658_0005
16/01/30 01:24:20 INFO mapreduce.Job: 

In [13]:
# examing the results
!hdfs dfs -cat /w261/hw3/hw3_1_output/part-00000

debt_collection_complaints	44372
mortgage_complaints	125752
other_complaints	142788


* <b>HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters</b>
Perform a word count analysis of this single record dataset using a Mapper and Reducer 
based WordCount (i.e., no combiners are used here) using user defined Counters to count up 
how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter,
 and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively.
 Please explain.
 Please use multiple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using 
a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined 
Counters to count up how many time the mapper and reducer are called. What is the value of 
your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, 
Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined 
Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your 
user defined Mapper Counter, and Reducer Counter after completing your word count job. 
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? 
Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and 
their frequency and their relative frequency. If there are ties please sort the tokens in 
alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [15]:
%%writefile hw3_2_input.txt
foo foo quux labs foo bar quux

Writing hw3_2_input.txt


In [16]:
# put the input file into hdfs
!hdfs dfs -put hw3_2_input.txt /w261/hw3

In [22]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")
for line in sys.stdin:    
    line = line.strip().lower()
    if line == '': continue
    for word in line.split(' '):
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


In [38]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# record reducer invokation
sys.stderr.write("reporter:counter:custom_counters,reducer_count,1\n")
from operator import itemgetter

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    if len(line.split('\t')) != 2: continue

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


In [28]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_2_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -D mapred.map.tasks=1 -D mapred.reduce.tasks=4 -mapper mapper.py -file /data/w261/hw3/mapper.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/hw3_2_input.txt -output /w261/hw3/hw3_2_output

rm: `/w261/hw3/hw3_2_output': No such file or directory
16/01/30 02:07:58 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob1960028508392768622.jar tmpDir=null
16/01/30 02:08:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:08:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:08:06 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:08:06 INFO mapreduce.JobSubmitter: number of splits:1
16/01/30 02:08:06 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/30 02:08:06 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
16/01/30 02:08:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454112721658_0008
16/01/30 02:0

In [32]:
!hdfs dfs -cat /w261/hw3/hw3_2_output/part-0000*

quux	2
foo	3
bar	1
labs	1


Since there is only one line and four distinct words, the optimal setting should be using 1 mapper and 4 reducers.  However, it seems that the default setting for Hadoop is to do 2 maper and 1 reducer.  If I specify mapper and reducer task counts in Hadoop Streaming parameter and 1 and 4, I now get 1 and 4 as specified in the homework problem.

* <b>HW3.2 continues</b>: Please use multiple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using 
a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined 
Counters to count up how many time the mapper and reducer are called. What is the value of 
your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [36]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")
for line in sys.stdin:    
    line = line.strip().lower()
    if line == '' or line.startswith("complaint id"): continue
    issue_text = line.split(',')[3]
    print '%s\t%s' % (issue_text, 1)

Overwriting mapper.py


In [39]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_2_1_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -D mapred.map.tasks=2 -D mapred.reduce.tasks=2 -mapper mapper.py -file /data/w261/hw3/mapper.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/Consumer_Complaints.csv -output /w261/hw3/hw3_2_1_output

16/01/30 02:28:36 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_2_1_output
16/01/30 02:28:38 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob1057362170451352704.jar tmpDir=null
16/01/30 02:28:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:28:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:28:46 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:28:46 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 02:28:46 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/30 02:28:46 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.m

The mapper and reducer counter are both 2, which is exactly what I specified in the parameters.  This is not always going to be the case though - we don't have direct control over how Hadoop is goint to split jobs.  The parameters is just a hint for the system.

* <b>HW3.2 continues</b>Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, 
Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined 
Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your 
user defined Mapper Counter, and Reducer Counter after completing your word count job.

In [35]:
%%writefile combiner.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,combiner_count,1\n")
from operator import itemgetter

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    if len(line.split('\t')) != 2: continue

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting combiner.py


In [36]:
!chmod +x mapper.py; chmod +x reducer.py; chmod +x combiner.py
!hdfs dfs -rm -r /w261/hw3/hw3_2_2_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -D mapred.map.tasks=2 -D mapred.reduce.tasks=2 -mapper mapper.py -file /data/w261/hw3/mapper.py -combiner combiner.py -file /data/w261/hw3/combiner.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/Consumer_Complaints.csv -output /w261/hw3/hw3_2_2_output
#!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -D mapred.map.tasks=2 -D mapred.reduce.tasks=2 -mapper mapper.py -file /data/w261/hw3/mapper.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/Consumer_Complaints.csv -output /w261/hw3/hw3_2_2_output

16/01/30 04:11:29 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_2_2_output
16/01/30 04:11:31 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/combiner.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob7026719206050507258.jar tmpDir=null
16/01/30 04:11:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 04:11:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 04:11:38 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 04:11:38 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 04:11:38 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/30 04:11:38 INFO Configuration.deprecation: mapred.map.tasks is deprecated. 

Combiner Counter: 4<br/>
Mapper Counter: 2<br/>
Reducer Counter: 2<br/>

* <b>HW3.2 Continue</b>: Using a single reducer: What are the top 50 most frequent terms in your word count analysis? 
Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in 
alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [37]:
!hdfs dfs -cat /w261/hw3/hw3_2_2_output/* | head -10

"account opening	16205
"making/receiving payments	3226
advertising and marketing	1193
arbitration	168
balance transfer	502
balance transfer fee	95
bankruptcy	222
can't contact lender	221
cash advance	136
cash advance fee	104
cat: Unable to write to output stream.


In [21]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import csv
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")

total_count = 0
# input comes from STDIN
for line in sys.stdin:
    line = line.strip().lower()
    if line == '' or line.startswith("complaint id"): continue
    lineArr = list(csv.reader(line.splitlines(), delimiter=',', quotechar='"'))[0]
    issue_text = lineArr[3]
    total_count += 1
    if issue_text.strip() == '': issue_text = 'blank'
    print '%s\t%s\t%s' % ("issue_words", issue_text, 1)

# print total word count from this mapper to calculate relative frequency
print '%s\t%s\t%s' % ("issue_words", "*", total_count)

Overwriting mapper.py


In [25]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# record reducer invokation
sys.stderr.write("reporter:counter:custom_counters,reducer_count,1\n")
total_count = 0
current_issue = None
current_count = 0
for line in sys.stdin:
    line = line.strip()
    if line == '': continue
    lineArr = line.split('\t')
    issue_text = lineArr[1]
    issue_count = int(lineArr[2])
    # aggregate total counts
    if issue_text == "*":
        total_count += issue_count
    else:
        if issue_text == current_issue:
            current_count += issue_count
        else:
            if current_issue != None and total_count != 0:
                print "%s\t%s\t%.6f" % (current_issue, current_count, float(current_count)/float(total_count))
            current_issue = issue_text
            current_count = issue_count
if current_issue != None and total_count != 0:
    print "%s\t%s\t%.6f" % (current_issue, current_count, float(current_count)/float(total_count))

Overwriting reducer.py


In [26]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_2_3_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar \
-D stream.num.map.output.key.fields=3 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k3nr -k2" \
-mapper mapper.py -file /data/w261/hw3/mapper.py \
-reducer reducer.py -file /data/w261/hw3/reducer.py \
-input /w261/hw3/Consumer_Complaints.csv \
-output /w261/hw3/hw3_2_3_output;

16/01/30 22:35:59 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_2_3_output
16/01/30 22:36:02 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob3516562719083472012.jar tmpDir=null
16/01/30 22:36:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 22:36:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 22:36:09 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 22:36:09 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 22:36:09 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/01/30 22:36:09 INFO Configuration.deprecation: mapred.text.key.comparato

Top 50 frequent items and their relative frequencies:

In [28]:
!hdfs dfs -cat /w261/hw3/hw3_2_3_output/part-00000 | sort -t$'\t' -k2,2nr -k1,1 | head -50

loan modification,collection,foreclosure	70487	0.225261
loan servicing, payments, escrow account	36767	0.117499
incorrect information on credit report	29069	0.092898
cont'd attempts collect debt not owed	17972	0.057435
account opening, closing, or management	16205	0.051788
deposits and withdrawals	10555	0.033732
communication tactics	8671	0.027711
application, originator, mortgage broker	8625	0.027564
disclosure verification of debt	7655	0.024464
billing disputes	6938	0.022172
other	6273	0.020047
problems caused by my funds being low	5663	0.018098
credit reporting company's investigation	4858	0.015525
managing the loan or lease	4560	0.014573
unable to get credit report/credit score	4357	0.013924
settlement process and costs	4350	0.013902
repaying your loan	3844	0.012285
problems when you are unable to pay	3821	0.012211
false statements or representation	3621	0.011572
improper contact or sharing of info	3489	0.011150
apr or interest rate	3431	0.010965
identity theft / fraud / embezzleme

Bottom 10 tokens and their counts and relative frequencies

In [29]:
!hdfs dfs -cat /w261/hw3/hw3_2_3_output/part-00000 | sort -t$'\t' -k2,2n -k1,1 | head -10

blank	4	0.000013
incorrect/missing disclosures or info	64	0.000205
charged bank acct wrong day or amt	71	0.000227
convenience checks	75	0.000240
payment to acct not credited	92	0.000294
balance transfer fee	95	0.000304
wrong amount charged or received	98	0.000313
cash advance fee	104	0.000332
received a loan i didn't apply for	118	0.000377
overlimit fee	127	0.000406
